# Finding alpha signals in insider trading

In this post, we’ll take a look at insider trading (the legal kind, not the illegal kind) and what information, if any, investors can draw from the required disclosures that insiders must make.

In [10]:
import fidap, pandas as pd, importlib
importlib.reload(fidap)

<module 'fidap' from 'C:\\Users\\User\\Google Drive\\Python\\fidap-notebooks\\fidap.py'>

## What is insider trading

Insider trading generally has a negative connotation of being illegal. However, that’s most often not the case - the vast majority of insider trading is perfectly legal. An “insider” is any individual that may have material nonpublic information about a company. Generally, the definition includes the following people - 

1. Company directors
2. Company officers
3. Any shareholder with >10% holdings
4. Often, corporate insiders will need to trade in the shares of the company. Whenever an insider transacts in the shares of the company, they must file a declaration with the SEC called a Form 4. A filing looks something like this - 

More detailed information about what these fields mean can be found here.


## Is there information in insider trading?

For investors, the key question is whether or not insider trades can be used as a signal. There are two key questions we need to ask ourselves - 

1. What is the reason behind the insider trade?
2. If the reason behind the insider trade is price-related (eg an opinion on whether the price will go up or down), are insiders accurate?

There could be a variety of reasons that insiders trade in a security beyond whether they think the price is going to go up or down. For company directors and officers, a large part of their compensation may be based in shares, and they may decide to liquidate those shares for diversification or for liquidity needs for other reasons, such as taxes or other purchases. Though less common, even insider buys may have motivations driven by requirements, such as the need to maintain a certain level of ownership.

However, if the motivation of the insider trade is indeed an opinion on the future trajectory of the price of the stock, then we may conclude that insiders have additional information about the future prospects of the company. 

Therefore, can we assume that insiders buying shares is a positive signal and insiders selling shares is a negative signal?



## Data exploration using Fidap

Let’s see what the data tells us. Fidap has a dataset around insider trades that covers all SEC filings. Here’s a link to the table details page. There are 24 columns and 11.9m rows. The columns include information on the date of the transaction and the date of the filing, the transaction amount, the owner, the underlying security, as well as several other details.

The columns are below - 



In [11]:
fidap.sql("""
    select * from meta_fields where "table" = 'insider_trades'
""")

,table,name,display_name,description,pd_type,pct_filled,uniqueness,mean,median
0,insider_trades,ticker,Ticker Symbol,The ticker is a unique identifer for an issuer...,object,1.000000,1.516270e-03,NaN,NaN
1,insider_trades,filingdate,Filing Date,The date the form was filed with the SEC.,datetime64[ns],1.000000,3.777049e-04,NaN,NaN
2,insider_trades,formtype,Form Type,"""The type of SEC form . Available options are ...",object,1.000000,6.958026e-07,NaN,NaN
3,insider_trades,issuername,Issuer Name,The name of the security issuer.,object,0.990162,1.494700e-03,NaN,NaN
4,insider_trades,ownername,Owner Name (Insider / Investor),The name of the owner.,object,1.000000,1.888571e-02,NaN,NaN
5,insider_trades,officertitle,Officer Title,Is the owner is an officer of the company the ...,object,0.606439,7.818271e-03,NaN,NaN
6,insider_trades,isdirector,Is Director?,Is the owner a Board Director? [Y]es or [N]o.,object,1.000000,2.319342e-07,NaN,NaN
7,insider_trades,isofficer,Is Officer?,Is the owner an officer of the company? [Y]es ...,object,1.000000,2.319342e-07,NaN,NaN
8,insider_trades,istenpercentowner,Is Ten Percent Owner?,Does the owner hold ten percent or more of the...,object,1.000000,2.319342e-07,NaN,NaN
9,insider_trades,transactiondate,Transaction Date,If there has been a transaction; the date of t...,datetime64[ns],0.687956,7.905477e-04,NaN,NaN


Over fifteen years that we have data in Fidap, we see a total 12 million transactions, covering 15,000 tickers, and a whopping $9 trillion in transaction volume. See the query below

In [4]:
fidap.sql("""
    select count(*) as count, sum(transactionvalue) as tvalue, 
    count(distinct(ticker)) as tickers, min(filingdate) from insider_trades
""")

,count,tvalue,tickers,min
0,11901766,9.274746e+12,15104,2005-01-03


Let's first explore the data and find some basic numbers here.

We can see these same numbers in the Fidap dashboard as well.

Let's see some of the individual rows here.

In [6]:
fidap.sql("""
    select * from insider_trades where ticker='STT'  order by filingdate desc limit 5
""")

,ticker,filingdate,formtype,issuername,ownername,officertitle,isdirector,isofficer,istenpercentowner,transactiondate,...,sharesownedfollowingtransaction,transactionpricepershare,transactionvalue,securitytitle,directorindirect,natureofownership,dateexercisable,priceexercisable,expirationdate,rownum
0,STT,2020-11-20,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-19,...,18304.0,68.59,27436.0,Common Stock,D,None,None,None,None,1.0
1,STT,2020-11-20,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,NaT,...,550.0,NaN,NaN,Common Stock,I,By domestic partner,None,None,None,2.0
2,STT,2020-11-17,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-15,...,19020.0,68.52,53514.0,Common Stock,D,None,None,None,None,1.0
3,STT,2020-11-17,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-16,...,18704.0,70.50,22278.0,Common Stock,D,None,None,None,None,2.0
4,STT,2020-11-17,4,STATE STREET CORP,PHELAN DAVID C,EVP; Gen Counsel and Secretary,N,Y,N,2020-11-15,...,96249.0,68.52,124843.0,Common Stock,D,None,None,None,None,1.0


We can actually get info on each of the columns from the metadata table. Let's take a look.

In [5]:
fidap.sql("""
    select * from insider_trades where ticker='AAPL' and securitytitle = 'Common Stock' limit 5
""")

,ticker,filingdate,formtype,issuername,ownername,officertitle,isdirector,isofficer,istenpercentowner,transactiondate,...,sharesownedfollowingtransaction,transactionpricepershare,transactionvalue,securitytitle,directorindirect,natureofownership,dateexercisable,priceexercisable,expirationdate,rownum
0,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,550892.0,NaN,NaN,Common Stock,D,None,None,None,None,1.0
1,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,323396.0,116.79,26569258.0,Common Stock,D,None,None,None,None,2.0
2,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-01,...,1008340.0,NaN,NaN,Common Stock,D,None,None,None,None,1.0
3,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-01,...,746603.0,116.79,30568264.0,Common Stock,D,None,None,None,None,2.0
4,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-02,...,517646.0,113.52,25991198.0,Common Stock,D,None,None,None,None,3.0


In [6]:
# need to link with tickers table
fidap.sql("""
    select ticker, count(*) as count, sum(transactionvalue) as tvalue from insider_trades
    where transactionvalue > 0 group by ticker order by tvalue desc limit 10
""")

,ticker,count,tvalue
0,NGBL,1001,1.425084e+12
1,CDNC,303,2.035737e+11
2,GGP,1325,8.086486e+10
3,PWRM,88,6.180867e+10
4,TELOZ,63,5.091514e+10
5,HLT,361,5.049469e+10
6,OMAG,69,4.371894e+10
7,SOYL,83,4.360486e+10
8,TMRK,191,4.332359e+10
9,QVDX,171,4.293356e+10


In [7]:
fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionvalue from insider_trades, tickers
    where insider_trades.ticker = tickers.ticker and insider_trades.transactionvalue > 100000 and insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
    limit 100
""")

,ticker,filingdate,transactionvalue
0,AAPL,2020-10-05,26569258.0
1,AAPL,2020-10-05,30568264.0
2,AAPL,2020-10-05,25991198.0
3,AAPL,2020-10-05,3241397.0
4,AAPL,2020-10-05,32193046.0
...,...,...,...
95,ETSY,2020-10-05,252450.0
96,ETSY,2020-10-05,255981.0
97,ETSY,2020-10-05,103337.0
98,ETSY,2020-10-05,286126.0


In [14]:
df = fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionshares,
    insider_trades.transactionvalue, daily.close, daily.fc_1m
    from insider_trades, tickers, daily
    where insider_trades.ticker = 'AAPL' and
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
""")

In [15]:
df

,ticker,filingdate,transactionshares,transactionvalue,close,fc_1m
0,AAPL,2015-06-01,70000.0,461300.0,32.634,-0.030153
1,AAPL,2015-06-01,-70000.0,9202900.0,32.634,-0.030153
2,AAPL,2015-06-26,-12697.0,1630930.0,31.688,-0.029696
3,AAPL,2015-06-26,-11388.0,1471443.0,31.688,-0.029696
4,AAPL,2015-07-21,-47170.0,6114175.0,32.688,-0.138430
...,...,...,...,...,...,...
433,AAPL,2015-03-06,-15806.0,2031703.0,31.650,-0.007899
434,AAPL,2015-03-06,-3400.0,437920.0,31.650,-0.007899
435,AAPL,2015-03-11,-2800.0,361116.0,30.560,0.037696
436,AAPL,2015-03-17,-20842.0,2575863.0,31.760,-0.018010


In [16]:
df['fc_1m'].mean()

0.02679334840309135

In [17]:
df[df['transactionshares'] > 0]['fc_1m'].mean()

0.024119978448000002

In [18]:
df[df['transactionshares'] < 0]['fc_1m'].mean()

0.02692471793405407

In [19]:
len(df[df['transactionshares'] > 0]['fc_1m'])

20

In [21]:
df2 = fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionshares,
    insider_trades.transactionvalue, daily.close, daily.fc_1m
    from insider_trades, tickers, daily
    where 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
""")

In [25]:
df2[df2['transactionshares'] > 0]['fc_1m'].mean()

0.011137640542707588

In [27]:
df2[df2['transactionshares'] < 0]['fc_1m'].mean()

0.010430338106194136

In [35]:
fidap.sql("""
    select avg(daily.fc_3m), avg(daily.fc_1m), count(daily.fc_3m)
    from insider_trades, tickers, daily
    where 
    insider_trades.transactionshares > 0 and 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock'
""")

,avg,avg,count
0,0.035062,0.011499,85430


In [36]:
fidap.sql("""
    select avg(daily.fc_3m), avg(daily.fc_1m), count(daily.fc_3m)
    from insider_trades, tickers, daily
    where 
    insider_trades.transactionshares < 0 and 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
""")

,avg,avg,count
0,0.029452,0.001935,206154


In [9]:
fidap.sql("""select * from fundamentals where ticker='URBN' and dimension = 'MRQ' order by calendardate desc""")

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
0,URBN,MRQ,2020-09-30,2020-10-31,2020-10-31,2020-12-11,-30894000.0,3.530675e+09,None,1.371561e+09,...,97786384.0,97784664.0,98583030.0,9.916,3.530675e+09,117705000.0,20914000.0,0.0,36.107,424663000.0
1,URBN,MRQ,2020-06-30,2020-07-31,2020-07-31,2020-12-11,-29203000.0,3.425278e+09,None,1.270966e+09,...,97779584.0,97778750.0,98104920.0,8.215,3.425278e+09,121292000.0,34486000.0,0.0,35.031,499749980.0
2,URBN,MRQ,2020-03-31,2020-04-30,2020-04-30,2020-12-11,-40925000.0,3.356204e+09,None,1.176928e+09,...,97777320.0,97910310.0,97910310.0,6.010,3.356204e+09,169054000.0,-60131000.0,0.0,34.278,513351000.0
3,URBN,MRQ,2019-12-31,2020-01-31,2020-01-31,2020-12-11,-28004000.0,3.315633e+09,None,1.053396e+09,...,97975344.0,97955864.0,98913630.0,11.940,3.315633e+09,104578000.0,20077000.0,0.0,33.848,414625980.0
4,URBN,MRQ,2019-09-30,2019-10-31,2019-10-31,2020-12-11,-29691000.0,3.320593e+09,None,1.113013e+09,...,97975344.0,97972864.0,98628170.0,10.079,3.320593e+09,114641000.0,20193000.0,0.0,33.893,401961000.0
5,URBN,MRQ,2019-06-30,2019-07-31,2019-07-31,2020-12-11,-37287000.0,3.138045e+09,None,1.000397e+09,...,97965010.0,99095560.0,99602464.0,9.711,3.138045e+09,105814000.0,21239000.0,0.0,31.667,374264000.0
6,URBN,MRQ,2019-03-31,2019-04-30,2019-04-30,2020-12-11,-30717000.0,3.251820e+09,None,1.139297e+09,...,103559360.0,104437456.0,105340144.0,8.277,3.251820e+09,101267000.0,10115000.0,0.0,31.137,491117980.0
7,URBN,MRQ,2018-12-31,2019-01-31,2019-01-31,2020-12-11,-27103000.0,2.160515e+09,None,1.202756e+09,...,107642280.0,107106650.0,108376710.0,10.540,2.160515e+09,104438000.0,28973000.0,0.0,20.172,816112000.0
8,URBN,MRQ,2018-09-30,2018-10-31,2018-10-31,2020-12-11,-32093000.0,2.197042e+09,None,1.248799e+09,...,109134680.0,108778480.0,110262880.0,8.950,2.197042e+09,103327000.0,20072000.0,0.0,20.197,793826000.0
9,URBN,MRQ,2018-06-30,2018-07-31,2018-07-31,2020-12-11,-26601000.0,2.158535e+09,None,1.201768e+09,...,108824690.0,108831400.0,110433840.0,9.119,2.158535e+09,104169000.0,25789000.0,0.0,19.834,771830000.0


In [10]:
fidap.sql("""select * from meta_fields where "table" = 'fundamentals'""")

,table,name,display_name,description,pd_type,pct_filled,uniqueness,mean,median
0,fundamentals,ticker,Ticker Symbol,[Entity] The ticker is a unique identifer for ...,object,1.000000,0.014239,NaN,NaN
1,fundamentals,dimension,Dimension,[Entity] The dimension field allows you to tak...,object,1.000000,0.000011,NaN,NaN
2,fundamentals,calendardate,Calendar Date,"""[Entity] The Calendar Date represents the nor...",datetime64[ns],1.000000,0.000046,NaN,NaN
3,fundamentals,datekey,Date Key,[Entity] The Date Key represents the SEC filin...,datetime64[ns],1.000000,0.003534,NaN,NaN
4,fundamentals,reportperiod,Report Period,[Entity] The Report Period represents the end ...,datetime64[ns],1.000000,0.001697,NaN,NaN
...,...,...,...,...,...,...,...,...,...
106,fundamentals,taxassets,Tax Assets,[Balance Sheet] A component of [Assets] repres...,float64,0.987689,0.059580,1.069366e+09,0.00
107,fundamentals,taxexp,Income Tax Expense,[Income Statement] Amount of current income ta...,float64,0.974903,0.152351,1.282391e+09,414000.00
108,fundamentals,taxliabilities,Tax Liabilities,[Balance Sheet] A component of [Liabilities] r...,float64,0.987735,0.073238,2.394779e+09,0.00
109,fundamentals,tbvps,Tangible Assets Book Value per Share,[Metrics] Measures the ratio between [Tangible...,float64,0.984484,0.151023,2.801606e+04,18.19
